# Python Practice Lecture 10 MATH 342W Queens College - Multivariate Linear Regression with the Hat Matrix
## Author: Amir ElTabakh
## Date: March 3, 2021

## Agenda:
* Multivariate Linear Regression with the Hat Matrix

## Multivariate linear regression with the Hat Matrix

First let's do the null model to examine what the null hat matrix looks like. In this exercise, we will see that $g_0 = \bar{y}$ is really the OLS solution in the case of no features, only an intercept i.e. $b_0 = \bar{y}$.

We'll load in the Boston Housing data.

In [1]:
# Lines below are just to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Importing dependencies
from sklearn import datasets
import pandas as pd
import numpy as np

# Load the Boston Housing dataset as bh
bh = datasets.load_boston()

# Initialize target variable
y = bh.target

# Create Boston Housing df
df = pd.DataFrame(data = bh.data, columns = bh.feature_names)

# Load the first 5 rows of df
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Let's build a linear model of just the intercept column.

In [4]:
# Importing dependencies
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score # RMSE, R^2

# Intitialize model
model_intercept = LinearRegression()

# define 1-vector
ones = [[1] for i in range(len(df))]

# convert to a numpy array
ones = np.asarray(ones, dtype=np.float64)

# Fit y on the 1-vector
model_intercept.fit(ones, y)

# get yhat
yhat = model_intercept.predict(ones)

# print first 20 predictions
yhat[0:20]

array([22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632])

In [3]:
# print the mean of y
print(np.mean(y))

22.532806324110677


Let's do a simple example of projection. Let's project $y$ onto the intercept column, the column of all 1's. What do you think will happen?

In [5]:
H = ones @ ones.transpose() / sum(ones**2)
H[1:5, 1:5]

array([[0.00197628, 0.00197628, 0.00197628, 0.00197628],
       [0.00197628, 0.00197628, 0.00197628, 0.00197628],
       [0.00197628, 0.00197628, 0.00197628, 0.00197628],
       [0.00197628, 0.00197628, 0.00197628, 0.00197628]])

In [6]:
# output shape of the Hat matrix
H.shape

(506, 506)

In [7]:
# Output unique values of Hat matrix
np.unique(H)

array([0.00197628])

In [7]:
# In fact
print(1 / 506)

0.001976284584980237


The whole matrix is just one single value for each element! What is this value? It's $\frac{1}{506}$ where 506 is $n$. So what's going to happen?

In [8]:
# Getting y projected on ones
y_proj_one = H @ y
y_proj_one[0:20]

array([22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632,
       22.53280632, 22.53280632, 22.53280632, 22.53280632, 22.53280632])

Projection onto the space of all ones makes the null model ($g = \bar{y}$). It's the same as the model of response = intercept + error, i.e. $y = \mu_y + \epsilon$. The OLS intercept estimate is clearly $\bar{y}$. Let's build a multivariate linear regression model with the `LinearRegression` sklearn module.

In [9]:
# intitialize model
model = LinearRegression(fit_intercept = True)

# fit
model.fit(df, y)

# print intercept b0
print(model.intercept_)

# print coefficients
print(model.coef_)

36.459488385089855
[-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01]


Now we'll do the same using our linear algebra.

First we'll add the intercept column, and then we'll convert the df to a NumPy array so that we can more easily use NumPy's linear algebra functionality.

In [10]:
# Let's get our b vec
X = df

# add intercept column
X.insert(0, 'INTERCEPT', [1 for i in range(len(X))])

# Convert X to numpy array (matrix)
X = X.to_numpy()

# linear algebra
Xt = X.transpose()
XtXinv = np.linalg.inv(Xt @ X)
b = XtXinv @ Xt @ y
b

array([ 3.64594884e+01, -1.08011358e-01,  4.64204584e-02,  2.05586264e-02,
        2.68673382e+00, -1.77666112e+01,  3.80986521e+00,  6.92224640e-04,
       -1.47556685e+00,  3.06049479e-01, -1.23345939e-02, -9.52747232e-01,
        9.31168327e-03, -5.24758378e-01])

We calculated the same intercept and coefficient values. Let's use the Hat matrix to calculate all predictions.

In [11]:
X.transpose()

array([[1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 1.0000e+00, 1.0000e+00,
        1.0000e+00],
       [6.3200e-03, 2.7310e-02, 2.7290e-02, ..., 6.0760e-02, 1.0959e-01,
        4.7410e-02],
       [1.8000e+01, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [1.5300e+01, 1.7800e+01, 1.7800e+01, ..., 2.1000e+01, 2.1000e+01,
        2.1000e+01],
       [3.9690e+02, 3.9690e+02, 3.9283e+02, ..., 3.9690e+02, 3.9345e+02,
        3.9690e+02],
       [4.9800e+00, 9.1400e+00, 4.0300e+00, ..., 5.6400e+00, 6.4800e+00,
        7.8800e+00]])

In [12]:
# get Hat matrix
H = X @ XtXinv @ Xt

# The `.dot()` method works fine
#H = X.dot(XtXinv.dot(Xt))

print(H.shape)

(506, 506)


In [13]:
# Calculate your predictions
yhat = H @ y
yhat[0:10]

array([30.00384338, 25.02556238, 30.56759672, 28.60703649, 27.94352423,
       25.25628446, 23.00180827, 19.53598843, 11.52363685, 18.92026211])

Can you tell this is projected onto a 13 dimensionsal space from a 506 dimensional space? Not really... but it is...

Now let's project over and over...

In [14]:
(H @ H @ H @ H @ H @H @ H @ H @ H @ y)[0:10]

array([30.00384338, 25.02556238, 30.56759672, 28.60703649, 27.94352423,
       25.25628446, 23.00180827, 19.53598843, 11.52363685, 18.92026211])

Same thing! Once you project, you're there, you can't project to another different space. That's the idempotency of $H$.

Let's make sure that it really does represent the column space of $X$. Let's try to project different columns of $X$:

In [15]:
# H @ intercept
print(X[0:5, 0])
print((H @ X[:, 0])[0:5])

[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]


In [16]:
# H @ CRIM
print(X[0:5, 1])
print((H @ X[:, 1])[0:5])

[0.00632 0.02731 0.02729 0.03237 0.06905]
[0.00632 0.02731 0.02729 0.03237 0.06905]


In [17]:
# H @ ZN
print(X[0:5, 2])
print((H @ X[:, 2])[0:5])

[18.  0.  0.  0.  0.]
[ 1.80000000e+01 -3.86579657e-13  5.09148279e-13  3.55937502e-13
 -7.48734408e-13]


In [18]:
# H @ INDUS
print(X[0:5, 3])
print((H @ X[:, 3])[0:5])

[2.31 7.07 7.07 2.18 2.18]
[2.31 7.07 7.07 2.18 2.18]


In [19]:
# H @ CHAS
print(X[0:5, 4])
print((H @ X[:, 4])[0:5])

[0. 0. 0. 0. 0.]
[-3.80251386e-15 -3.23092247e-15  2.74086309e-15  1.52568930e-15
 -5.36853548e-15]


We can calculate the residual error using the Hat matrix as well.

In [23]:
e = y - yhat
e[0:5]

array([-6.00384338, -3.42556238,  4.13240328,  4.79296351,  8.25647577])

In [24]:
I = np.identity(len(X))
e = (I - H) @ y
e[0:5]

array([-6.00384338, -3.42556238,  4.13240328,  4.79296351,  8.25647577])

Let's do that projection over and over onto the complement of the column space of $X$:

In [25]:
((I - H) @  (I - H) @ (I - H) @ (I - H) @ (I - H) @ (I - H) @ y)[0:5]

array([-6.00384338, -3.42556238,  4.13240328,  4.79296351,  8.25647577])